In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [11]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Num GPUs Available:  1


In [4]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.2964 - accuracy: 0.9138
Epoch 2/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.1440 - accuracy: 0.9574
Epoch 3/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.1051 - accuracy: 0.9686
Epoch 4/5
60000/60000 [==============================] - 4s 62us/sample - loss: 0.0891 - accuracy: 0.9731
Epoch 5/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.0757 - accuracy: 0.9761
10000/1 - 0s - loss: 0.0427 - accuracy: 0.9748


[0.07957178548835218, 0.9748]

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15436107990429176862
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3149044121
locality {
  bus_id: 1
  links {
  }
}
incarnation: 381306334379592232
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [10]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [11]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [13]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [14]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [16]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.038991693407297134, 정확도: 98.79444122314453, 테스트 손실: 0.06001625582575798, 테스트 정확도: 98.30667114257812
에포크: 2, 손실: 0.0344114787876606, 정확도: 98.93071746826172, 테스트 손실: 0.061043038964271545, 테스트 정확도: 98.33428955078125
에포크: 3, 손실: 0.030595898628234863, 정확도: 99.0491714477539, 테스트 손실: 0.061056043952703476, 테스트 정확도: 98.36750030517578
에포크: 4, 손실: 0.027639193460345268, 정확도: 99.13963317871094, 테스트 손실: 0.062255654484033585, 테스트 정확도: 98.38333129882812
에포크: 5, 손실: 0.02533702552318573, 정확도: 99.20883178710938, 테스트 손실: 0.06324975937604904, 테스트 정확도: 98.39500427246094
에포크: 6, 손실: 0.023282695561647415, 정확도: 99.27197265625, 테스트 손실: 0.06500054150819778, 테스트 정확도: 98.40909576416016
에포크: 7, 손실: 0.02156313695013523, 정확도: 99.32611083984375, 테스트 손실: 0.06636383384466171, 테스트 정확도: 98.41333770751953
에포크: 8, 손실: 0.020105380564928055, 정확도: 99.37154388427734, 테스트 손실: 0.06824995577335358, 테스트 정확도: 98.41384887695312
에포크: 9, 손실: 0.018739111721515656, 정확도: 99.41404724121094, 테스트 손실: 0.0691857784986496, 테스트 정확도